# 多様な制約を一般化した強化学習のフレームワーク

参考：
* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)
* [A Survey of Constraint Formulations in Safe Reinforcement Learning](https://arxiv.org/abs/2402.02025)

CMDPでは期待値の意味での制約を考えますが，それ以外にも制約の実現方法はあります．
今回はいろいろなタイプのSafe RLを見てみましょう．

表記：
* 価値：$V_{r, h}^\pi(s):=\mathbb{E}_\pi\left[\sum_{h^{\prime}=h}^H \gamma_r^{h^{\prime}} r\left(s_{h^{\prime}}, a_{h^{\prime}}\right) \mid s_h=s\right]$
* 初期状態での期待値：$V_c^\pi(\rho):=\mathbb{E}_{s \sim \rho}\left[V_{c, 0}^\pi(s)\right]$
* $V_{\max }:=\frac{1-\gamma_r^H}{1-\gamma_r}$
* ベルマン最適作用素：$\mathcal{T}_h(Q)(s, a):=\mathbb{E}\left[r\left(s_h, a_h\right)+\gamma_r V_Q\left(s_{h+1}\right) \mid s_h=s, a_h=a\right]$, where $V_Q(s):=\max _{a \in \mathcal{A}} Q(s, a)$

また，方策空間は決定的であることを仮定します（解けるのか？）

## いろいろなSafe RL

---

**1. 期待値制約**

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad V_c^\pi(\rho) \leq \xi \text {. }
$$

これは扱いやすいのでよく使われます．
特にラグランジュ形式にすると報酬とコストが合体できるので解きやすいです．

---

**2. Safe RL with state constraints**

* [Recovery RL: Safe Reinforcement Learning with Learned Recovery Zones](https://arxiv.org/abs/2010.15920)など

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{E}_\pi\left[\sum_{h=0}^H \gamma_c^h \mathbb{I}\left(s_h \in S_{\text {unsafe }}\right)\right] \leq \xi
$$

危険な状態に突入してしまうことを避ける問題設定です．
ロボティクスの応用などでよく出てくるっぽいですね．
ちなみにこの問題は$c(s, a):=\mathbb{I}\left(s \in S_{\text {unsafe }}\right)$とすれば，**1**に含まれます．

---

**3. Joint chance constraint**

* [Safe Reinforcement Learning with Chance-constrained Model Predictive Control](https://arxiv.org/abs/2112.13941)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{P}_\pi\left[\bigvee_{h=0}^H s_h \in S_{\text {unsafe }}\right] \leq \xi,
$$

$h=0$から$H$までを総合して，危険な状態にいる確率が一定値以下であることを保証します．
制御の問題でたまに出てくるみたいです．
これは実際解きづらいので，次のゆるい問題を解くことがあります．

$$
\mathbb{P}_\pi\left[\bigvee_{h=1}^H s_h \in S_{\text {unsafe }}\right] \leq \mathbb{E}_\pi\left[\sum_{h=1}^H \mathbb{I}\left(s_h \in S_{\text {unsafe }}\right)\right]
$$

この意味で，**2**は**3**よりもゆるい問題です．

---

**4. Expected Instantaneous Safety Constraint
with Time-variant Threshold**

* [OptLayer - Practical Constrained Optimization for Deep Reinforcement Learning in the Real World](https://arxiv.org/abs/1709.07643)など

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{E}_\pi\left[c\left(s_h, a_h\right)\right] \leq \xi_h, \forall h \in[H]
$$

あんまり見ないですが，各ステップで制約を考える設定です．
多分普通の問題よりは解きやすいのかも？

---

**追記**

* $\xi$が固定の場合，$\xi=\min_{s, a} c(s, a)$とすれば，１に含まれますね．なので，明らかに１のほうが難しいです（これは嘘ですね．Cumulativeだと足されちゃうのでやっぱり別物です）．
    * ただし，制約を$H$個増やせばInstantaneousも解くことができると思います．各ステップで閾値が変わる場合も同様．

---

また，
$$
\eta_h:=\gamma_c^{-h} \cdot\left(\xi-\sum_{h^{\prime}=0}^{h-1} \gamma_c^{h^{\prime}} c\left(s_{h^{\prime}}, a_{h^{\prime}}\right)\right), \forall h \in[H]
$$
を使って$\mathbb{E}_\pi\left[c\left(s_h, a_h\right) \leq \eta_h\right]$を考えると，これは**1**と等価っぽいです．

**TODO: これは$\eta_h$が方策に依存してしまうので，等価ではないのでは？各エピソードで方策が変わるとすると，これはNon-stationary MDPを解くことになるかもしれない．**


---
**5. Almost surely safe RL**

* [Saute RL: Almost Surely Safe Reinforcement Learning Using State Augmentation](https://arxiv.org/abs/2202.06558)で導入されたっぽい

$$
\max _\pi V_r^\pi(\rho) \quad \text { subject to } \quad \mathbb{P}_\pi \left[\sum_{h=1}^H \gamma_c^h c\left(s_h, a_h\right) \leq \xi \right]=1,
$$

これは期待値よりも強い制約を課す問題です．
TODO: これNP困難になりそうだけど大丈夫か？Mean-variance MDPを考えるとこれNP困難になりそうな気がする．

---

**6. Almost Surely Instantaneous Safety
Constraint with Time-invariant Threshold**

* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)
* [End-to-End Safe Reinforcement Learning through Barrier Functions for Safety-Critical Continuous Control Tasks](https://arxiv.org/abs/1903.08792)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{P}_\pi\left[c\left(s_h, a_h\right) \leq \xi\right]=1, \forall h \in[H]
$$

各ステップ全部で制約を満たすことを考え，しかもAlmost surelyであることを要求します．
制御工学におけるバリア関数やリアプノフ関数とかで使われるっぽい？

---

**7. Almost Surely Instantaneous Safety Constraint with Time-variant Threshold**

* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{P}_\pi\left[c\left(s_h, a_h\right) \leq \xi_h\right]=1, \forall h \in[H]
$$

これは複数の問題を一般化できる点で便利っぽい．
* 明らかにこれは**6**の一般化
* また，
$$
\eta_h:=\gamma_c^{-h} \cdot\left(\xi-\sum_{h^{\prime}=0}^{h-1} \gamma_c^{h^{\prime}} c\left(s_{h^{\prime}}, a_{h^{\prime}}\right)\right), \forall h \in[H]
$$
を使って$\mathbb{P}_\pi\left[c\left(s_h, a_h\right) \leq \eta_h\right]=1$を考えると，これは**5**と等価っぽいです．

**TODO: これは$\eta_h$が方策に依存してしまうので，等価ではないのでは？各エピソードで方策が変わるとすると，これはNon-stationary MDPを解くことになるかもしれない．**
* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)で言及されているが，$\eta_h$が$h$になるまでわからないような問題にすれば大丈夫っぽい．
* でも事前に固定されているか，それとも$\pi$に応じて変化するか，で問題が異なる気がする．解くべきは後者だと思われるが，この論文は前者しか考えてないだろ．

---

**8. 他の問題**

* [Policy Gradients with Variance Related Risk Criteria](https://arxiv.org/abs/1206.6404)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \operatorname{Var}\left[V_r^\pi(\rho)\right]<\xi \text {. }
$$

分散が一定以下になるような制約を課す問題です．これはMean-variance MDPみたいにNP困難になりそうな気がする．

後は自然言語を利用するアプローチもあるみたいです．

TODO: Shieldingの話


## 問題７を解くアルゴリズム

参考：
* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)

上の論文で問題７を解くアルゴリズムが提案されているので，見てみましょう．
論文に合わせて問題を再定義します．

---

**GSE problem**

$b_h$を閾値とします．次を考えましょう．

$$
\max _\pi V_r^\pi \quad \text { subject to } \quad \operatorname{Pr}\left[g\left(s_h, a_h\right) \leq b_h \mid \mathcal{P}, \pi\right]=1, \quad \forall h \in[1, H]
$$

ここで，$b_h$は$h$になるまでわからないとします（多分これは方策に応じて変化するAdversarialな値にしないといけない気がするが...　一応論文通りに進んでみよう．）．

また，方策は決定的であるとします．このとき，上の制約は$g(s_h, a_h)\leq b_h$と同じです．

---

次の仮定を導入します．

1. $\operatorname{Pr}\left[g\left(s_h, a_h\right) \leq b_h-\zeta \mid \mathcal{P}, \pi^{\star}\right]=1 \quad \forall h \in [1, H]$を満たす$\zeta \in \mathbb{R}_{> 0}$が存在する
2. $\mathcal{P}(s_1 \mid s, \hat{a}) = 1$が成立するemergency stop actionが存在する．つまり，これは初期状態に戻す行動．また，この行動はsafetyが満たされる可能性が十分大きくない場合にのみ実行可能である．
3. $\mu: \mathcal{S} \times \mathcal{A} \to \mathbb{R}$を安全性の期待値を推定する関数とします．$\mathrm{t}|g(s, a)-\mu(s, a)| \leq \Gamma(s, a)$を確率$1-\delta$以上で満たす関数$\Gamma : \mathcal{S} \times \mathcal{A} \to \mathbb{R}$が存在することを仮定します．
    * TODO: これ意味わかんないな．$\mu$が関数なら確率とかないよ．$\mu$がなんかのProbability spaceに従う確率変数ならまだわかる．
    * 多分，$\hat{\mu}_{s, a}$を確率変数として，適切な$\sigma$-代数を設定しないといけない．
    * 読む限り$g(s, a)$はdeterministicなので，そもそも$\mu$を考える意味がない．全体的によくわからない．

---

アルゴリズムについて見ていきましょう．

![MASE](figs/MASE-algorithm.png)

次の特徴があります：

1. $g(s, a) \in[\mu(s, a) \pm \Gamma(s, a)]$が$1-\delta$以上で成立するので，安全な行動の集合を
$$\mathcal{A}_h^{+}:=\left\{a \in \mathcal{A} \mid \min \left\{1, \mu\left(s_h, a\right)+\Gamma\left(s_h, a\right)\right\} \leq b_h\right\}$$
として表現します．
TODO: ここで$1$でclipするのもよくわかんないな．$\eta$がGeneralizeするような形で取られているなら，これは$1$でclipしたらダメだろ．

2. $\mathcal{A}_h^+$が空であれば，stop action $\hat{a}$を実行して良いとします．
    * このとき，$\widehat{r}\left(s_h, a_h\right)=-\frac{c}{\min _{a \in \mathcal{A}} \Gamma\left(s_{h+1}, a\right)}$として，$\left(s_h, a_h, \widehat{r}\left(s_h, a_h\right), s_{h+1}\right)$をデータセットに加えます
3. そうでなければ，$\left(s_h, a_h, r\left(s_h, a_h\right), s_{h+1}\right)$をデータセットに加えます

直感：

例えば$\Gamma = 0$のときを考えましょう．
このとき$g$の情報は完全に得られています．
そして，$\mathcal{A}_h^+$が空である場合はその状態に行くことが危険なので，$-\infty$の報酬が課され，その場所に行くことはなくなります．

TODO: これやっぱり問題あるな．例えば$s$がちょうど危険になってしまう方策でそこに訪れると，そこには$-\infty$の報酬が割り当てられるので，他の方策でそこに訪れることが不可能になり，明らかに最適方策にたどり着けないはず．やっぱり$b_h$は方策に依存するようにしなければならない．

---

**Generalized Linear Bellman Completenessでの応用**

まず次のGeneralized Linear Model (GLM) を定義します．

**GLM**

* $d \in \mathbb{Z}_{> 0}$を特徴次元とし，$\mathbb{B}^d:=\left\{\mathrm{x} \in \mathbb{R}^d:\|\mathrm{x}\|_2 \leq 1\right\}$を$\mathbb{R}^d$上の$l_2$ボールとします．
* 既知の特徴マップ関数$\phi : \mathcal{S} \times \mathcal{A} \to \mathbb{R}^d$および既知のリンク関数$f:[-1,1] \rightarrow[-1,1]$を考えます
* このとき，$\mathcal{F}:=\{(s, a) \mapsto f(\left\langle\phi_{s, a}, \theta\right\rangle): \theta \in \mathbb{B}^d\}$なる関数の集合をGLMと呼ぶことにします．

Assumption 5.2に書いてますが，Link関数にも適切な仮定を置きます．

このとき，次のGL-CMDPを考えることができます．

---

**GL-CMDP**

$$
\mathcal{F}_{\text {up }}:=\left\{(s, a) \rightarrow \min \left\{V_{\max }, f\left(\left\langle\phi_{s, a}, \theta\right\rangle\right)+\alpha\left\|\phi_{s, a}\right\|_A\right\}: \theta \in \mathbb{B}^d, 0 \leq \alpha \leq \alpha_{\text {max }},\|A\|_{\text {op }} \leq 1\right\}
$$
とします．これはつまり線形関数にリンク関数を適用＆正則化項が入った形なので，より一般的です．

任意の$1 \leq h < H$および$u \in \mathcal{F}_{\text {up }}$について，

$$
\mathcal{T}_h(u) \in \mathcal{F} \text { and } g \in \mathcal{F}
$$

の場合にGL-CMDPと呼びます．つまり，$\mathcal{F}_{up}$をベルマン作用素に突っ込んだ結果は一般線形関数で表現可能であり，また，コスト関数は一般線形関数で表現可能です．ここで，特徴ベクトルは（多分）シェアされていることに注意しましょう．
